In [1]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_gene_sequence(genome_location, 
                      species, 
                      genome_file, 
                      scaffold,
                      gene_start,
                      gene_end,
                      complement,
                      output_location):

    
    print("Getting Gene")
    list_of_folders = os.listdir(output_location)
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"0.Temp" not in list_of_folders:
        os.mkdir(f"{output_location}/0.Temp")

#     subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
#     print(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end}')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{output_location}/0.Temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    
    genome = SeqIO.parse(f"{output_location}/0.Temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)
def make_output_folder(output_location):
    list_of_folders = os.listdir(output_location)
    if "2.Final_output" in list_of_folders:
        subprocess.run(f'rm -r "{output_location}/2.Final_output"', shell = True, stderr = subprocess.DEVNULL)
    os.mkdir(f"{output_location}/2.Final_output")

def write_exon_output(output_location,species,exon,exon_output):
    
    
    list_of_folders_in_final_output = os.listdir(f"{output_location}/2.Final_output")
    if species not in list_of_folders_in_final_output:
        os.mkdir(f"{output_location}/2.Final_output/{species}")
    
    with open(f"{output_location}/2.Final_output/{species}/{exon}.fa", 'w') as out_file:
        out_file.write(exon_output)

In [2]:
import os
from Bio import SeqIO
import subprocess

family_group = "3.Satyrine"

output_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}"
make_output_folder(output_location)
genome_location = "/mnt/f/Genomes_2023-12-26"
list_of_species = os.listdir(f"{output_location}/1.Blast_result")
if 'desktop.ini' in list_of_species:
    list_of_species.remove("desktop.ini")
# print(list_of_species)
# list_of_species = ["Heliconius_charithonia"]
# list_of_species = ["Maniola_hyperantus"]
error_exons = ''
for species in list_of_species:
    print(species)
    genome_file = get_genome_file(genome_location,species)
    print(genome_file)
    previous_coordinate_start = ''
    previous_scaffold = ''
    with open(f"{output_location}/1.Blast_result/{species}/final_coordinates.csv", 'r') as coordinate_final_file:
        coordinate_file_list = coordinate_final_file.readlines()
    
    for i in range(1,len(coordinate_file_list)):
        
        coordinate_line_split = coordinate_file_list[i].split(",")
        
            
#         print(coodinate_line_split[5])
        if coordinate_line_split[5] == "N":
            if previous_coordinate_start == '':
                previous_coordinate_start = coordinate_line_split[2]
                previous_scaffold = coordinate_line_split[1]
            complement = coordinate_line_split[4]
            start,stop = coordinate_line_split[2],coordinate_line_split[3]
            print(start,stop)
            query_name = coordinate_line_split[6]
            scaffold = coordinate_line_split[1]
            exon = f"Exon_{query_name.split('_')[-1]}"
            left_oh, right_oh = query_name.split('_')[-6], query_name.split('_')[-4]
            print(left_oh, right_oh)
#             print(complement)
            if complement == "0":
                if start < previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            elif complement == "1":
                if start > previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            else:
                print("ERROR")
            
        elif coordinate_line_split[5] == "Y":
            query_name = coordinate_line_split[6]
            exon = f"Exon_{query_name.split('_')[-1]}"
            error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            continue
            
        gene_sequence = get_gene_sequence(genome_location, 
                                          species, 
                                          genome_file, 
                                          scaffold,
                                          start,
                                          stop,
                                          complement,
                                          output_location)
        
       
        current_exon = query_name.split('_')[-1]
        if current_exon == "1":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()
            print(translated)
            if "*" in translated:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            else:
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
        
                write_exon_output(output_location,species,exon,exon_output)
        elif current_exon == "16":
            if complement == 1:
                translated = gene_sequence[:-int(left_oh)].reverse_complement.translate()
                print(gene_sequence)
            else:
                translated = gene_sequence[int(left_oh):].translate()
                print(gene_sequence)

            if "*" in translated[:-1]:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                print(translated)
            else:
                
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
                write_exon_output(output_location,species,exon,exon_output)
#         assert False
        else:
            exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
            print(exon_output)

            write_exon_output(output_location,species,exon,exon_output)
        
#         assert False
        previous_coordinate_start = start
        previous_scaffold = scaffold
    error_exons += "\n\n"
#     print(coordinate_file_list)
#     break
coordinate_file = ''
print(error_exons)
with open(f"{output_location}/error_exons.txt", 'w') as error_out_file:
    error_out_file.write(error_exons)

Coenonympha_glycerion
GCA_963855885.1_ilCoeGlyc1.1_genomic.fna
10535487 10535583
0 1
Getting Gene


/home/saurav/.local/lib/python3.8/site-packages/Bio/Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


MEWVLRSPQIHNTFSDLGFALPDGYHINDSCN
>Coenonympha_glycerion_Exon_1_OY979629.1_10535487_10535583_left_0_right_1
ATGGAGTGGGTACTCAGAAGTCCCCAAATTCACAACACATTCAGCGACCTCGGATTTGCACTTCCAGATGGATATCACATAAACGATAGTTGCAACT
10533898 10534448
2 0
Getting Gene
>Coenonympha_glycerion_Exon_2_OY979629.1_10533898_10534448_left_2_right_0
CCGCTTTGGAGACCATTCTTCAAAACCTTGTGACGGAAGACAAATACTCGCGCTCATATCGCAGAAGTATTAGCTTCGgacaaaatatcaaaaaagACCTTATACCTGTGTTAATCCACACCAAAGATGATAAAACCGTTGAGTTGTTAATCAAAATATTGGTTAATTTAACGATTCCGGTGGAGTGTTTACTTTCGTTGGAAACAGTTTCACAAACCGAAATCGGGAGACACGCGATTTTAGAAATAAACAATTTGCTCTCTTGCACCAAAATTGCGTTTACCGATCACCATGTGGCAAAAGTCGTTGTGGACTTGTTGAAGAAGAACGCCGAGAATGATGAAAAGGGAAAATTGACGACAGAAAATTGTACGAATATAAGCAACACCCTTTTGTTTCTGCGTAATTTGCTCCACATACCGGAGGACATTGATTGTCACAGCCCGACGTACAGTGGCCCACCTCATACGATTCAGAACCAGATATTATGGAATTTATTCAGCCACAGTATCGACAAACTACTCATAAAATTGATGACAATTCCGGATGCG
10533204 10533357
0 1
Getting Gene
>Coenonympha_glycerion_Exon_3_OY979629.1_10533204_10533357_left_0_right_1
ATAAACTGGGGCGTGACAATGGTGC

>Elymnias_hypermnestra_Exon_8_JACTOM010000070.1_5056799_5056893_left_0_right_2
gATCTGATCGTGACAAATCACAATCTTTTATTGTTCATGGACGGTGTGATGCAAAATGAAAAGGCCAGCCCCAATATGGTTGACCATATAAAACa
5056354 5056552
1 0
Getting Gene
>Elymnias_hypermnestra_Exon_9_JACTOM010000070.1_5056354_5056552_left_1_right_0
GTTTTGTTCTGCTGAGATTATGTATCAGTATGGTTTGCTTTTGGAGGATTACGAAGGAAATGGTGAATTCGTTAATGActgtatatttacaataatgcATCACGTGGCAGGGGAGTTGGGTAGCCCGATTTCATTGTTTCAACCAAAGATCCTCAGAATCTTTACAGCTATATGCAAATCTGAATTTGAAATATGTGAC
5055623 5055834
0 2
Getting Gene
>Elymnias_hypermnestra_Exon_10_JACTOM010000070.1_5055623_5055834_left_0_right_2
GATTGGACAGATCTCATAGAGTATGTGATAAATACATTTGTAAAGAAACCATATAATCTACAGAGTTCGATAAAACTCTTTCTGGATGATAGTAAATTCGATGATCCCAGGTTAAAAGAAATTGTGGTTCCCGCGAACAGTATAGATAAGCTTATCCCGGAAGTGTCCACAGTACAGGACAAAGAGCCTTCTGATCCCAATAAGTCGGTCGG
5054659 5055125
1 1
Getting Gene
>Elymnias_hypermnestra_Exon_11_JACTOM010000070.1_5054659_5055125_left_1_right_1
atgGACAGATGATGAATTGTCAACGCTAAGTTGGAATTACATGCAATGCAATAGCAGATCAGATGTCGTTGGGG

Erebia_ligea
GCA_917051295.2_ilEreLige1.2_genomic.fna
11529184 11529283
0 1
Getting Gene
MEWVLRSPQIHNTFSNLGFPLPDGGYHINENCN
>Erebia_ligea_Exon_1_OU785219.1_11529184_11529283_left_0_right_1
ATGGAGTGGGTGCTCAGAAGTCCTCAAATCCATAATACCTTCAGCAACCTCGGATTTCCACTTCCAGATGGAGGGTATCACATAAACGAAAATTGCAACT
11529385 11529935
2 0
Getting Gene
>Erebia_ligea_Exon_2_OU785219.1_11529385_11529935_left_2_right_0
CCGCTTTAGAGTCCATTTTTCAAAACCTTACTACCGAAGATAAGTTCTTACGTACATATCGCAGAAGTATCAGCTTTgggcaaaatattaaaaaagaccTTAAACCTCTATTAATCAATACCAAGGATGATAAAACAACAGAGctgttaattaaaatattcgtcAATTTGACCATTCCGGTGGAATGTTTACTATCATTAGAATCCATATCAAGGACCGATATAGGGAGGCATACCATAttcgaaataaataatttactctCTGCGACAAAAATCGCGCTAACTGATCATCgtgttacaaaaataattattgaatttttaaagaaaaattctGAAAATGAACAAAAAGGAAAGCTGTCTACAGAACAGTGTACAAACATAACCAACACTCTTTTAATTCTGCGTAATATACTACATATACCAGAGGATGTAAATTGCCACAGTCCGACGTACAGTGGTCCACCCCACACAATTCAAAATCAGTTATTGTGGAATTTATTTAGTCAGAGTATTGacaaagttctcataaaattgATGACGATTCCAGATGCG
11530528 11530681
0 1
Getting Gene
>Erebia_ligea_Exon_3_OU7

...

MEWVLRSPQIHNTFSNLGFPLPDGGYHINENCN
>Maniola_hyperantus_Exon_1_NC_048536.1_8589746_8589845_left_0_right_1
ATGGAGTGGGTTCTCCGAAGTCCCCAAATCCATAACACTTTCAGCAACCTTGGATTTCCACTTCCAGATGGAGGGTATCACATAAACGAAAATTGCAACT
8589953 8590503
2 0
Getting Gene
>Maniola_hyperantus_Exon_2_NC_048536.1_8589953_8590503_left_2_right_0
CTGCTTTAGAGTCAATTTTTCACAACCTCATAACAGAAGATAAACATTTGCGAACATATCGCAGAAGTATCAGCTTTggacaaaatattaataaagacCTTGTACCTCTGTTAATTAACACCAAGGATGGTAAAACAATAGAGctgttaattaaaatattcgtCAATTTAACCATTCCGGTAGAATGCTTACTGTCTTTAGATTCAGTTTCAAGGACCGATATAGGGAGGCATACtatatttgaaataaataatttgcttTCTTCGACAAAAATCGCACTTACTGATCATCGTGTAACAAAAGttattattgaatttttaaagaaGAATTCCGAAAATGAGCAAAAGGGGAAGCTGTCTCCGGAACAGTGTACAAATATAAGCAACAGCCTTTTAATTCTGCGTAACATACTACATATACCAGAAGATGTCAATTGCCACAGTCCAACATATAGTGGTCCACCTCACACAATTCAAAATCAGTTATTATGGAATTTATTTAGTCAAAGTATTGACAAAGTACTCATAAAATTAATGACAATTCCTGATGCG
8591017 8591170
0 1
Getting Gene
>Maniola_hyperantus_Exon_3_NC_048536.1_8591017_8591170_left_0_right_1
ATAAACTGGGGAGTGACAATGGTGCAATTAATCGCCT

9414492 9414592
0 2
Getting Gene
>Maniola_jurtina_Exon_8_NC_060029.1_9414492_9414592_left_0_right_2
GATCTGGTCGTGACTAATCATATTCTTCTGATGTTTATGGACAACGTCATGAAATTGCCCGATTACAAAGGCACTGGCACTATGGTTGACCATATAAGACA
9413413 9413611
1 0
Getting Gene
>Maniola_jurtina_Exon_9_NC_060029.1_9413413_9413611_left_1_right_0
ATTTTGCTCCCCAGAAATAATGTATCAATACGGTCTACTCTTAGAGGATTACGAGGGAAACGGTGAATTCGTGAATGACTGCGTATTTACACTGATGCATCACGTTGGCGGCGAGCTGGGAAGCTTAATTTCACTGTTCCAGCCGAAAATCTTGAAGACATTCACATTAATTTGGAAGTCGGAGTTCGAAATATGTGAC
9412761 9412975
0 2
Getting Gene
>Maniola_jurtina_Exon_10_NC_060029.1_9412761_9412975_left_0_right_2
GATTGGTCAGATCTAATAGAGTATGTAATAAACACATTCATAAAGAAGCCATACAAGCTACAAAGTTCAGTACAAGTCCGCTTGGACGCGGAGATGTTTGATAACGATACGAGGTTGAAAGAAATACTAATGTCTGCTGAAAACACGGAGGAATTCATGCCAGAAAAATTGTTACCGCAAGAAGAAAAATCGGTCCAACCCAATAATTCTGCTGG
9411816 9412291
1 1
Getting Gene
>Maniola_jurtina_Exon_11_NC_060029.1_9411816_9412291_left_1_right_1
ttGGACCGAAGATGAGCTATCATCACTGAGTTGGAATTTCTTGCAATGCAACGATTTACCAGATGTGATCGGGGAAATGG

CGTGTGCGAGACGGCGTCCGTGGCGTCGGACCTCACGCGCATGTACGTGTCGGACGAGGACGACAAGCCCGAGCCGGCGCGCCCCCGCCCTGCGCACCCCGCACACCCCCACCCCCACCCCCACCCCGCCGCGCGCTCCGCCGCCGGGAAACGACAGTTCTCATCGCGTTTTTAA
>Melanargia_galathea_Exon_16_OV049862.1_11734421_11734595_left_1_right_0
CGTGTGCGAGACGGCGTCCGTGGCGTCGGACCTCACGCGCATGTACGTGTCGGACGAGGACGACAAGCCCGAGCCGGCGCGCCCCCGCCCTGCGCACCCCGCACACCCCCACCCCCACCCCCACCCCGCCGCGCGCTCCGCCGCCGGGAAACGACAGTTCTCATCGCGTTTTTAA
Oeneis_ivallda
GCA_029955525.1_ilOenChry1.0.hap2_genomic.fna
14422564 14422663
0 1
Getting Gene
MEWVQQSSQIHSTFSNLGYSLPNGGYHINENCN
>Oeneis_ivallda_Exon_1_JARPMR010000002.1_14422564_14422663_left_0_right_1
ATGGAGTGGGTTCAGCAAAGTTCCCAAATCCATAGCACCTTCAGCAACCTCGGATATTCTCTTCCCAATGGAGGATATCATATAAACGAGAATTGCAACT
14423066 14423616
2 0
Getting Gene
>Oeneis_ivallda_Exon_2_JARPMR010000002.1_14423066_14423616_left_2_right_0
CTGCTTTGGAGTCTATTCTTGAAAACCTTGCAACGGAAGACAAATATTTGCGTACCTATCGCCGAAGTATCAGCTTTGGGCAAAATATTAAGAAAGACCTTATACCTTTATTAATTAACACCAAAGATGATAAAGCTATAGAGCTGTTAACGAAAATATTC

>Pararge_aegeria_Exon_7_NC_053182.1_11814208_11814399_left_0_right_0
GTTGTAACAGCTATCCGAGAATTCATTCAAGCAATTGATACGTACAAGAAATCAGCGCAAATCTGCAATAACGACAAAGAACTGCTTGACAACTTGCAACTAAAAATATGCGAGACAGATGAATTGCGTTCGCTTCTAGTGCTGCTTTTAAGGTATTATAACCCGAAGTACCAAACGAAGCAATATCTGCAG
11813638 11813738
0 2
Getting Gene
>Pararge_aegeria_Exon_8_NC_053182.1_11813638_11813738_left_0_right_2
gATCTTGTGGTGACAAATCACAATCTTCTGGTGTTAACGGACGTTACCATGAAGTTGCCGGAGTACAAGGGAACTGACACTATAGCTGATCACATTAGACA
11811871 11812069
1 0
Getting Gene
>Pararge_aegeria_Exon_9_NC_053182.1_11811871_11812069_left_1_right_0
ATTTTGTGCCCCAGAAATGATGTATCAGTACGGTCTACTTTTGGAGGACTACCAGATAAACGGTGAATTTGTAAACGACTGCATATTTACACTGATGCATCACGTCGGCGGGGATATGGGAAGCCCGATTTCACTATTCCAGCCAAACATCGTGAGAACATTTAAAGCAATTCTGAAGTCGGACTTTGAAATATGTGAC
11809840 11810042
0 2
Getting Gene
>Pararge_aegeria_Exon_10_NC_053182.1_11809840_11810042_left_0_right_2
GATTGGACAGATCTAATTGAGTATGTAATGAATAAATACGTAAAGAAGCCCCTAAAAATGAGAAGCGCCTTACTACGCTCCCTAGGCATAGACATTAATGGGGACGATATAAGGTTTCAAAAC

CGTGTGCGACACGGCGTCCGTGGCGTCCGACCTCACGCGCATGTACGTGTCCGACGAGGACGACAAGCCGGAGCCGCTGCTGCTGGCCGAGCGCTCGCTGAAGGCGGCGCGACACGAGCTCGCCGACATGGCGGACTCCGGCGCCAGCAAACGGCCGCGCGTCGACTTCTCACCGCCATTCTAA
>Bicyclus_anynana_Exon_16_NC_069085.1_10730856_10731039_left_1_right_0
CGTGTGCGACACGGCGTCCGTGGCGTCCGACCTCACGCGCATGTACGTGTCCGACGAGGACGACAAGCCGGAGCCGCTGCTGCTGGCCGAGCGCTCGCTGAAGGCGGCGCGACACGAGCTCGCCGACATGGCGGACTCCGGCGCCAGCAAACGGCCGCGCGTCGACTTCTCACCGCCATTCTAA

























